\* 이 글에서는 `Scheme`을 이용하여 설명하겠습니다.

__내가 그의 이름을 불러주었을 때__


__그는 나에게로 와서__


__꽃이 되었다.__

( 김춘수의 시 '꽃' 중에서 )

위의 시에서 '그의 이름'은 '어떤 타자'를 지칭하는 의미입니다.

그런데 만약, '-그의 이름-이라고 말해봐'하고 말하면 거기서 -그의 이름-은 __글자 그대로__ -그의 이름-이 됩니다.

이처럼 '글자 그대로'를 데이터로 다룰 때 그것을 __글자 데이터( Symbolic data )__라고 합니다.

여기서는 글자 데이터를 가지고 만든 식을 미분하는 코드를 작성하겠습니다.

먼저, 미분을 할 때 사용하는 규칙을 정리하면,

### 1. $\frac{dc}{dx} = 0$

### 2. $\frac{dx}{dx} = 1$

### 3. $\frac{d(u+v)}{dx} = \frac{d(u)}{dx} + \frac{d(v)}{dx}$

### 4. $\frac{d(uv)}{dx} = u(\frac{dv}{dx}) + v(\frac{du}{dx})$

3.과 4.에 대한 코드는 모듈로 아래와 같습니다.

```scheme
;; Judge exp is a number?(not symbol?) and num = exp?
(define (=number? exp num)
  (and (number? exp) (= exp num)))


;; 3. make-sum
(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2)) (+ a1 a2))
        (else (list '+ a1 a2))))


;; 4. make-product
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2)
        ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2))))
```

위의 모듈은 들어온 데이터가 

1) 숫자인지( 숫자가 아니면 글자로 인식 ) 판단

2) 1.과 2.에 해당하는지

3) 그게 아니면 3.과 4.의 정의대로 글자식으로 만듭니다.

( \* 이 코드는 아래에서 더 보완합니다. )

그다음, 대수식에서 사용할 기본 프로시저를 코드 없이 설명만 나열하겠습니다.



```
( Referenced by SICP )


(variable? e) Is e a variable?

(same-variable? v1 v2) Are v1 and v2 the same variable?

(sum? e) Is e a sum?

(addend e) Addend of the sum e.

(augend e) Augend of the sum e.

(make-sum a1 a2) Construct the sum of a1 and a2.

(product? e) Is e a product?

(multiplier e) Multiplier of the product e.

(multiplicand e) Multiplicand of the product e.
```

\* `augend`는 덧셈수에서 첫째 마디, `addend`는 덧셈수에서 둘째 마디입니다. 

예를들어, `a1 + a2`이면 `a1`은 `augend`, `a2`는 `addend`입니다.

\* 마찬가지로 `multiplicand`는 곱셈수에서 첫째 마디, `multiplier입니다`는 곱셈수에서 둘째 마디입니다. 

예를들어, `m1 * m2`이면 `m1`은 `multiplicand`, `m2`는 `multiplier`입니다.

이제, 여기서 구현해볼 미분 규칙을 정리하면,

### 5. $\frac{d(u^n)}{dx} = nu^{n-1}(\frac{du}{dx})$

### 6. $\frac{d(u+v+w+...)}{dx} = \frac{d(u)}{dx} + \frac{d(v)}{dx} + \frac{d(w)}{dx} +\ ...$

### 7. $\frac{d(uvw...)}{dx} = uw...(\frac{dv}{dx}) + vw...(\frac{du}{dx}) + uv...(\frac{dw}{dx})$

5.를 구현하기 위해서는, 위에서 만든 `make-sum`, `make-product`처럼 지수를 계산하는 `make-exponentiation`이 필요합니다.


그리고 지수인지 확인하는 `exponentiation?`, 식에서 밑과 지수를 분리하는 `base`, `exponent`도 만들면,

```scheme
(define (base x)
  (cadr x))


(define (exponent x)
  (caddr x))


(define (exponentiation? x)
  (and (pair? x) (eq? (car x) '**)))  


(define (make-exponentiation base exponent)
  (cond ((=number? exponent 0) 0)
        ((=number? exponent 1) 1)
        (else (list '** base exponent))))
```

5.의 규칙을 살펴보면,

$\frac{d(u^n)}{dx} = nu^{n-1}(\frac{du}{dx})$

세 수의 곱으로 되어있음을 알 수 있습니다. 따라서 이것은 `make-product`로 표현할 수 있습니다.

`make-product`$(n, u^{n-1}, \frac{du}{dx}$)

이런 방법으로 계속 연산자를 함수화하면,

`make-product`$(n,$ `make-exponentiation`$(u^{n-1})$, `derivation`$(\frac{du}{dx})$)

다만, 코드의 구현상 make-product는 2개의 마디(항)를 가지므로, 

`make-product`(`make-product`$(n,$ `make-exponentiation`$(u^{n-1})$), `derivation`$(\frac{du}{dx})$))

위와 같이 표현하도록 하겠습니다.

이에 대한 핵심 부분을 구현하면,

```scheme
...
(cond ...
    ((exponentiation? exp)
     (if (number? (exponent exp))
         (make-product (make-product (exponent exp)
                                     (make-exponentiation (base exp)
                                                          (- (exponent exp) 1))
                                     )
                       (deriv (base exp) var))
         (make-product (make-product (exponent exp)
                                     (make-exponentiation (base exp)
                                                          (list '- (exponent exp) 1))
                                     )
                       (deriv (base exp) var))))
                       ...
         ```

여기서 '지수가 숫자인가, 문자인가'에 따라 어디까지를 계산하고 어디까지를 글자식으로 표현할지 결정합니다.

( \* 그리고 이 부분이 구현을 다소 중복되게 만들었습니다. 이 부분에 대한 함수화를 거치면 코드를 더 간결하게 만들 수 있을 것입니다. )

몇가지 경우에 대해 검증해보면,